In [5]:
import pandas as pd
import numpy as np
from itertools import product
import math
import warnings

warnings.filterwarnings('ignore')

In [6]:
GCMs = ['CanESM5', 'GFDL-ESM4', 'MPI-ESM1-2-LR', 'BCC-CSM2-MR', 'IPSL-CM6A-LR']
SSPs = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
years = np.arange(2000, 2021)

In [7]:
choices = list(product(SSPs, GCMs, years))
num = len(choices)

In [14]:
for i in range(num):
    ssp, gcm, year = choices[i]
    file_final = './all_data/hist_%s.csv' % year
    all_covars = pd.read_csv(file_final)
    index = all_covars.isna().any(axis=1)
    all_covars = all_covars[~index].reset_index(drop=True)
    index = all_covars['target_23'] != 0
    target_num = len(all_covars[index])
    index = all_covars['target_23'] == 0
    samples_0 = all_covars[index]
    index = all_covars['target'] == 1
    samples_1 = all_covars[index]
    df = pd.DataFrame(all_covars['region'].value_counts(normalize=True))
    df['proportion'] = df['proportion'] * target_num
    df['proportion'] = df['proportion'].apply(math.ceil)
    for k in range(20):  # for 20 models
        df_list = [samples_1]
        for region in df.index:
            index = samples_0['region'] == region
            num = df.loc[region, 'proportion']
            df_list.append(samples_0[index].sample(num, replace=False))
        all_samples = pd.concat(df_list).reset_index(drop=True)
        all_samples.to_csv('./train_samples/%s_%02d.csv' % (year, k), index=0)
    break